<a href="https://colab.research.google.com/github/cateto/python4NLP/blob/main/topic_modeling/LSA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers','footers','quotes'))
document = dataset.data
len(document)

11314

In [10]:
document[1]

"\n\n\n\n\n\n\nYeah, do you expect people to read the FAQ, etc. and actually accept hard\natheism?  No, you need a little leap of faith, Jimmy.  Your logic runs out\nof steam!\n\n\n\n\n\n\n\nJim,\n\nSorry I can't pity you, Jim.  And I'm sorry that you have these feelings of\ndenial about the faith you need to get by.  Oh well, just pretend that it will\nall end happily ever after anyway.  Maybe if you start a new newsgroup,\nalt.atheist.hard, you won't be bummin' so much?\n\n\n\n\n\n\nBye-Bye, Big Jim.  Don't forget your Flintstone's Chewables!  :) \n--\nBake Timmons, III"

In [11]:
print(dataset.target_names)

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [12]:
news_df = pd.DataFrame({'document':document})
 # 특수 문자 제거
news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]"," ")
 # 길이가 3이하인 단어는 제거 (영어 특수)
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
 # 전체 단어에 대한 소문자 변환
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())

In [13]:
news_df['clean_doc'][1]

'yeah expect people read actually accept hard atheism need little leap faith jimmy your logic runs steam sorry pity sorry that have these feelings denial about faith need well just pretend that will happily ever after anyway maybe start newsgroup atheist hard bummin much forget your flintstone chewables bake timmons'

In [17]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [20]:
from nltk.corpus import stopwords
 # 영어 불용어를 불러온다.
stop_words = stopwords.words('english')
# 토큰화
tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split())
# 불용어에 없으면 리스트에 추가
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])

In [21]:
print(tokenized_doc[1])

['yeah', 'expect', 'people', 'read', 'actually', 'accept', 'hard', 'atheism', 'need', 'little', 'leap', 'faith', 'jimmy', 'logic', 'runs', 'steam', 'sorry', 'pity', 'sorry', 'feelings', 'denial', 'faith', 'need', 'well', 'pretend', 'happily', 'ever', 'anyway', 'maybe', 'start', 'newsgroup', 'atheist', 'hard', 'bummin', 'much', 'forget', 'flintstone', 'chewables', 'bake', 'timmons']


In [24]:
 #역토큰화
 detokenized_doc = []
 for i in range(len(news_df)):
   t = ' '.join(tokenized_doc[i])
   detokenized_doc.append(t)

news_df['clean_doc'] = detokenized_doc

In [25]:
news_df['clean_doc'][1]

'yeah expect people read actually accept hard atheism need little leap faith jimmy logic runs steam sorry pity sorry feelings denial faith need well pretend happily ever anyway maybe start newsgroup atheist hard bummin much forget flintstone chewables bake timmons'

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer

# 상위 1000개의 단어를 보존
vectorizer = TfidfVectorizer(stop_words='english', max_features=1000, max_df = 0.5, smooth_idf=True)

X = vectorizer.fit_transform(news_df['clean_doc'])
X.shape # TF-IDF 행렬의 크기 확인

(11314, 1000)

In [29]:
from sklearn.decomposition import TruncatedSVD
svd_model = TruncatedSVD(n_components=20, algorithm='randomized', n_iter=100, random_state=122)
svd_model.fit(X)
len(svd_model.components_)

20

In [30]:
terms = vectorizer.get_feature_names() # 단어 집합 상위 1000개가 저장됨.

def get_topics(components, feature_names, n=5):
  for idx, topic in enumerate(components):
    print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(5)) for i in topic.argsort()[:-n -1:1]])

In [31]:
get_topics(svd_model.components_,terms)

Topic 1: [('stephanopoulos', 0.00198), ('char', 0.00254), ('jpeg', 0.00404), ('contrib', 0.00434), ('null', 0.00501), ('stream', 0.00515), ('angeles', 0.00642), ('vancouver', 0.00653), ('entry', 0.00679), ('russia', 0.00691), ('committee', 0.00695), ('processing', 0.00744), ('matthew', 0.00747), ('default', 0.00748), ('march', 0.00753), ('mission', 0.00754), ('resource', 0.00771), ('genocide', 0.0078), ('institute', 0.00783), ('satellite', 0.00789), ('resources', 0.00802), ('lunar', 0.00827), ('volume', 0.00832), ('provides', 0.00832), ('armenia', 0.00834), ('soviet', 0.00839), ('turks', 0.0084), ('xterm', 0.00849), ('misc', 0.00856), ('export', 0.0086), ('soldiers', 0.00864), ('independent', 0.00875), ('muslim', 0.00875), ('nuclear', 0.00885), ('postscript', 0.00886), ('archive', 0.00889), ('agencies', 0.0089), ('supported', 0.0089), ('usenet', 0.00896), ('developed', 0.00906), ('copies', 0.00909), ('additional', 0.00909), ('table', 0.00914), ('passed', 0.00918), ('remote', 0.0092), (